In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import Series
import numpy as np
import h5py
from ast import literal_eval
import time
import json
import pickle
import matplotlib.pyplot as plt

C:\Users\sukan\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [48]:
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [50]:
from gensim import models, matutils

C:\Users\sukan\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Open file containing list of all restaurant business IDs

with open('C:/Users/sukan/Documents/TUM/Courses/Winter 2018/Data Mining Praktikum/Yelp/restaurant_ids.pkl', 'rb') as f:
    restaurant_ids = pickle.load(f)
f.close()

In [3]:
# Open Reviews File

with open('C:/Users/sukan/Documents/TUM/Courses/Winter 2018/Data Mining Praktikum/Yelp/dataset/yelp_dataset/yelp_academic_dataset_review.json', encoding="utf8") as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 
    
review_data = pd.DataFrame(data)
json_file.close()

In [4]:
# Extract only reviews of restaurants

restuarant_review_data = review_data.loc[review_data['business_id'].isin(restaurant_ids)]

In [6]:
restuarant_review_data.head()

,index,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g


In [5]:
# Reset index of restuarant_review_data

restuarant_review_data = restuarant_review_data.reset_index()

In [11]:
# user_restaurant_reviews = restuarant_review_data.groupby('user_id')['text'].apply(lambda x: x.join(' '))
restuarant_review_data.head()

,index,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
1,1,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
2,2,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
3,3,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
4,4,Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food. The steak is amazing. ...,0,msQe1u7Z_XuqjGoqhB0J5g


In [39]:
user_restaurant_reviews = pd.DataFrame(columns=['user_id', 'reviews', 'review_ids', 'avg_stars', 'avg_useful'])

In [25]:
len(restuarant_review_data)

3654797

In [29]:
relevant_reviews = restuarant_review_data[restuarant_review_data['user_id'].groupby(restuarant_review_data['user_id']).transform('size')>=10]

In [31]:
# Percentage of Relevant User Review Rows

len(relevant_reviews)/len(restuarant_review_data)*100

46.27698884507129

In [26]:
user_ids = set(list(restuarant_review_data[restuarant_review_data['user_id'].groupby(restuarant_review_data['user_id']).transform('size')>=10].user_id))

In [23]:
# user_groups
# for user_group in user_groups:
#     size_of_user_group = user_group.index
#     print(size_of_user_group)
#     break

In [28]:
user_restaurant_reviews

,user_id,reviews,review_ids,avg_stars,avg_useful


In [13]:
# user_ids = set(list(restuarant_review_data.user_id))

In [27]:
no_of_users = len(user_ids)
print (no_of_users)

64410


In [41]:
user_reviews_list = []
cnt = 0
for user_id in user_ids:
    rows = relevant_reviews[relevant_reviews.user_id == user_id]
    no_of_reviews_for_this_user = len(rows)
    reviews = ' '.join(rows.text)
    review_ids = list(rows.review_id)
    avg_stars = sum(rows.stars) / no_of_reviews_for_this_user
    avg_useful = sum(rows.useful) / no_of_reviews_for_this_user
    user_reviews_list.append(reviews)
    user_restaurant_reviews.loc[cnt] = [user_id, reviews, review_ids, avg_stars, avg_useful]    
    cnt = cnt + 1
    if cnt % 500 == 0:
        print ('completed ' + str(cnt))

completed 500
completed 1000
completed 1500
completed 2000
completed 2500
completed 3000
completed 3500
completed 4000
completed 4500
completed 5000
completed 5500
completed 6000
completed 6500
completed 7000
completed 7500
completed 8000
completed 8500
completed 9000
completed 9500
completed 10000
completed 10500
completed 11000
completed 11500
completed 12000
completed 12500
completed 13000
completed 13500
completed 14000
completed 14500
completed 15000
completed 15500
completed 16000
completed 16500
completed 17000
completed 17500
completed 18000
completed 18500
completed 19000
completed 19500
completed 20000
completed 20500
completed 21000
completed 21500
completed 22000
completed 22500
completed 23000
completed 23500
completed 24000
completed 24500
completed 25000
completed 25500
completed 26000
completed 26500
completed 27000
completed 27500
completed 28000
completed 28500
completed 29000
completed 29500
completed 30000
completed 30500
completed 31000
completed 31500
completed 32

In [42]:
user_restaurant_reviews.head()

,user_id,reviews,review_ids,avg_stars,avg_useful
0,79zwPS4BKiYbaCTjcjTPng,Ate here on Sunday at 3pm with my daughter and...,"[vC16EzYAGbpwlD_BYsApYA, Mbm5Ukg8CQmTUgImINt6h...",3.333333,0.083333
1,Y7019L7fz3l156_AICqhDg,We wanted to take advantage of their happy hou...,"[RmZGikoPmVQ0Rf_CrhcTTg, MFkKYDmYZkCDjZS7Wic6b...",3.210526,0.631579
2,REigeuE57-4ZYneT7_ADpA,Complimentary:\nSalad appetizer - very good\nN...,"[i0oLkJKc8PaZgT27LW0uVw, fJ_3iNXd_6pFGSJZwkx9U...",3.617647,0.264706
3,3T0UkEcsc_RNKrQVwG8gyQ,A world-class addition to the Charlotte restau...,"[MCxtRMb9Nsgym61OTipwkQ, b2WOanISecyKa4r_IiasB...",3.200000,2.600000
4,wirvylhlC43_ACfKpUDYYw,Never failed my tummy!! When I am very hungry ...,"[qvewfaKNA4u0NphupttaUA, a_BKPLCNu_vQXCwyXYud8...",3.300000,0.000000


In [54]:
len(user_restaurant_reviews)

64410

In [84]:
# user_reviews_list[:2]

In [47]:
wnl = WordNetLemmatizer()


class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [49]:
PUNCT = list(string.punctuation)

In [ ]:
vectorizer_1 = CountVectorizer(stop_words =
                     ENGLISH_STOP_WORDS.union(PUNCT),
                     ngram_range = (1,2),
                     max_df = 0.7, min_df = 30, tokenizer = LemmaTokenizer())

print ('Vectorization done.')

vectorized_user_reviews_1 = vectorizer_1.fit_transform(user_reviews_list)
feature_names_user_1 = vectorizer_1.get_feature_names()

print ('Fit transform done.')


corpus_user_reviews_1 = matutils.Sparse2Corpus(vectorized_user_reviews_1.transpose())

print ('Corpus created.')

id2word = {identifier: word for word, identifier in vectorizer_1.vocabulary_.items()}
user_reviews_lda_1 = models.LdaModel(corpus=corpus_user_reviews_1, num_topics=10, minimum_probability=0.03, id2word=id2word, passes=10)

print ('LDA done')

user_reviews_lda_1.print_topics()

Vectorization done.
Fit transform done.
Corpus created.


In [62]:
PUNCT

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [65]:
# feature_names_user_1

In [64]:
user_reviews_lda_1

[(0,
  '0.005*"server" + 0.004*"minute" + 0.004*"said" + 0.004*"asked" + 0.004*"``" + 0.004*"customer" + 0.004*"\'\'" + 0.003*"took" + 0.003*"2" + 0.003*"told"'),
 (1,
  '0.008*"\'\'" + 0.008*"``" + 0.005*"\'re" + 0.004*"--" + 0.004*"\'d" + 0.004*"\'ll" + 0.003*"think" + 0.002*"bit" + 0.002*"bar" + 0.002*"doe"'),
 (2,
  '0.008*"bar" + 0.006*"beer" + 0.005*"amazing" + 0.005*"happy" + 0.004*"friend" + 0.004*"hour" + 0.004*"spot" + 0.004*"option" + 0.004*"wine" + 0.003*"favorite"'),
 (3,
  '0.037*"le" + 0.026*"et" + 0.020*"la" + 0.015*"à" + 0.013*"pour" + 0.011*"est" + 0.010*"pa" + 0.010*"que" + 0.009*"je" + 0.009*"une"'),
 (4,
  '0.007*"pizza" + 0.005*"excellent" + 0.005*"burger" + 0.004*"wife" + 0.004*"quality" + 0.004*"star" + 0.004*"location" + 0.003*"sandwich" + 0.003*"area" + 0.003*"atmosphere"'),
 (5,
  '0.011*"vega" + 0.005*"buffet" + 0.004*"la" + 0.004*"strip" + 0.003*"la vega" + 0.003*"sushi" + 0.003*"flavor" + 0.003*"rice" + 0.003*"favorite" + 0.003*"meat"'),
 (6,
  '0.008*"dis

[(0,
  '0.005*"server" + 0.004*"minute" + 0.004*"said" + 0.004*"asked" + 0.004*"customer" + 0.003*"took" + 0.003*"2" + 0.003*"told"'), - CUSTOMER SERVICE
 (1,
  0.005*"\'re" + 0.004*"\'d" + 0.004*"\'ll" + 0.003*"think" + 0.002*"bit" + 0.002*"bar" + 0.002*"doe"'),
 (2,
  '0.008*"bar" + 0.006*"beer" + 0.005*"amazing" + 0.005*"happy" + 0.004*"friend" + 0.004*"hour" + 0.004*"spot" + 0.004*"option" + 0.004*"wine" + 0.003*"favorite"'), - BAR
 (3,
  '0.037*"le" + 0.026*"et" + 0.020*"la" + 0.015*"à" + 0.013*"pour" + 0.011*"est" + 0.010*"pa" + 0.010*"que" + 0.009*"je" + 0.009*"une"'),
 (4,
  '0.007*"pizza" + 0.005*"excellent" + 0.005*"burger" + 0.004*"wife" + 0.004*"quality" + 0.004*"star" + 0.004*"location" + 0.003*"sandwich" + 0.003*"area" + 0.003*"atmosphere"'), - LOCATION, AMBIENCE
 (5,
  '0.011*"vega" + 0.005*"buffet" + 0.004*"la" + 0.004*"strip" + 0.003*"la vega" + 0.003*"sushi" + 0.003*"flavor" + 0.003*"rice" + 0.003*"favorite" + 0.003*"meat"'), - FLAVOR OF FOOD
 (6,
  '0.008*"dish" + 0.005*"dessert" + 0.004*"flavor" + 0.004*"served" + 0.003*"bread" + 0.003*"cheese" + 0.003*"steak" + 0.003*"sweet" + 0.003*"wine" + 0.003*"cooked"'), - DISHES SERVED
 (7,
  '0.007*"cheese" + 0.005*"fry" + 0.004*"sandwich" + 0.004*"pizza" + 0.004*"burger" + 0.004*"flavor" + 0.003*"hot" + 0.003*"potato" + 0.003*"rice" + 0.003*"fried"'),  - FAST FOOD
 (8,
  '0.007*"dish" + 0.006*"rice" + 0.005*"noodle" + 0.005*"soup" + 0.004*"beef" + 0.004*"taste" + 0.004*"meat" + 0.003*"pork" + 0.003*"quite" + 0.003*"fried"'),  - ASIAN FOOD
 (9,
  '0.005*"wa n\'t" + 0.005*"amazing" + 0.004*"friend" + 0.004*"\'re" + 0.004*"think" + 0.004*"recommend" + 0.004*"super" + 0.003*"overall" + 0.003*"sushi" + 0.003*"tried"')] - OVERALL

In [82]:
topic_labels = ['customer service', 'none', 'bar', 'none', 'location & ambience', 'flavor of food', 'dishes served', 'fast food', 'asian food', 'overall']

In [66]:
EXTRA_PUNCT = ["'", "``", "\'"]
USELESS_WORDS = ["'ll", "'re", "'d", "\'re", "\'d", "\'ll", "le", "et", "la", "à", "est", "pa", "que", "je", "une", "wa", "n\'t"]

In [ ]:
vectorizer_2 = CountVectorizer(stop_words =
                     ENGLISH_STOP_WORDS.union(PUNCT).union(EXTRA_PUNCT).union(USELESS_WORDS),
                     ngram_range = (1,2),
                     max_df = 0.7, min_df = 30, tokenizer = LemmaTokenizer())

print ('Vectorizer created done.')




Vectorizer created done.


In [75]:
# vectorizer_2.stop_words

In [78]:
# vectorized_user_reviews_2 = vectorizer_2.fit_transform(user_reviews_list)
# feature_names_user_2 = vectorizer_2.get_feature_names()

# print ('Fit transform done.')

In [77]:
# vectorized_user_reviews_2

In [ ]:
corpus_user_reviews_2 = matutils.Sparse2Corpus(vectorized_user_reviews_2.transpose())

print ('Corpus created.')

id2word = {identifier: word for word, identifier in vectorizer_2.vocabulary_.items()}
user_reviews_lda_2 = models.LdaModel(corpus=corpus_user_reviews_2, num_topics=10, minimum_probability=0.03, id2word=id2word, passes=10)

print ('LDA done')

user_reviews_lda_2.print_topics()

In [76]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [79]:

# pass text from corpus to my LDA model
lda_corpus_1 = user_reviews_lda_1[corpus_user_reviews_1]
# changed type of corpus to csc
csc2_corpus = matutils.corpus2csc(lda_corpus_1).toarray().transpose()
# appended values from categorical features to my csc corpus
modified_csc2corpus = np.hstack((csc2_corpus, category_array))
# standard scaled the entire array due to the presence of the categorical features
scaled_corpus = scaler.fit_transform(modified_csc2corpus)

NameError: name 'category_array' is not defined

In [80]:
scaled_corpus = scaler.fit_transform(csc2_corpus)

In [81]:
scaled_corpus

array([[ 2.1038652 , -0.3108877 , -0.67270138, ...,  0.92950857,
        -0.34257105, -1.08844395],
       [ 1.16133097, -0.87872236,  1.13167644, ...,  0.84831194,
        -0.59502574, -1.08844395],
       [-1.02191027, -0.87872236, -0.94815191, ..., -0.40684996,
         3.79392082, -0.37468799],
       ...,
       [-1.02191027, -0.05164344, -0.62029238, ...,  2.27681498,
        -0.59502574, -1.08844395],
       [ 1.6548947 , -0.87872236, -0.94815191, ..., -0.95536032,
        -0.59502574,  0.38672352],
       [-1.02191027, -0.87872236,  0.44735035, ...,  0.07214417,
        -0.26361071,  2.10550835]])

In [83]:

# user_df contained the value of each bar for each of the features in my dimensionally reduced space
userdf_data = {label: values for label, values in zip(topic_labels, scaled_corpus.T)}
userdf_final = pd.DataFrame(userdf_data)
userdf_final.head()

,customer service,none,bar,location & ambience,flavor of food,dishes served,fast food,asian food,overall
0,2.103865,-0.073027,-0.672701,-0.359825,-0.603586,-0.676167,0.929509,-0.342571,-1.088444
1,1.161331,-0.073027,1.131676,-0.276274,-0.603586,-0.304102,0.848312,-0.595026,-1.088444
2,-1.021910,-0.073027,-0.948152,-0.227499,-0.603586,1.257419,-0.406850,3.793921,-0.374688
3,0.918140,-0.073027,0.004389,0.403429,-0.603586,0.951429,-0.955360,-0.595026,-1.088444
4,0.209307,-0.073027,-0.948152,-0.638397,-0.603586,-0.676167,-0.955360,1.512181,2.733130


In [86]:
len(user_restaurant_reviews)

64410

In [87]:
user_restaurant_reviews['user_interest'] = userdf_final.idxmax(axis=1).values

In [88]:
user_restaurant_reviews.head()

,user_id,reviews,review_ids,avg_stars,avg_useful,user_interest
0,79zwPS4BKiYbaCTjcjTPng,Ate here on Sunday at 3pm with my daughter and...,"[vC16EzYAGbpwlD_BYsApYA, Mbm5Ukg8CQmTUgImINt6h...",3.333333,0.083333,customer service
1,Y7019L7fz3l156_AICqhDg,We wanted to take advantage of their happy hou...,"[RmZGikoPmVQ0Rf_CrhcTTg, MFkKYDmYZkCDjZS7Wic6b...",3.210526,0.631579,customer service
2,REigeuE57-4ZYneT7_ADpA,Complimentary:\nSalad appetizer - very good\nN...,"[i0oLkJKc8PaZgT27LW0uVw, fJ_3iNXd_6pFGSJZwkx9U...",3.617647,0.264706,asian food
3,3T0UkEcsc_RNKrQVwG8gyQ,A world-class addition to the Charlotte restau...,"[MCxtRMb9Nsgym61OTipwkQ, b2WOanISecyKa4r_IiasB...",3.200000,2.600000,dishes served
4,wirvylhlC43_ACfKpUDYYw,Never failed my tummy!! When I am very hungry ...,"[qvewfaKNA4u0NphupttaUA, a_BKPLCNu_vQXCwyXYud8...",3.300000,0.000000,overall


In [96]:
user_restaurant_reviews[user_restaurant_reviews.user_interest=='asian food']['reviews'].values[0]

'Complimentary:\nSalad appetizer - very good\nNoodle appetizer - excellent\n\nVolcano roll - amazing\nSpicy medallions - ok, a bit filling\nSashimi - very good quality\n\nBlack sesame ice cream - good Lots of variety, always get lobster and chicken and duck congee which are both delicious. Came here for summerlicious. \n\nEscargots - in tomato sauce, was ok, I prefer it with a creamy or cheesy dressing.\n\nBeef carpaccio - delicately balanced flavours, delicious.\n\nSeafood pasta - had a good rose sauce, not too salty and not too bland, just right.\n\nProfiteroles were very good.\n\nOverall ok-good food, some dishes winning out over others. The butter chicken was good. Also had side dish of cauliflower and garlic naan, both were also good. Satisfies a butter chicken craving. Had the Buttermilk calamari which was delicious, very unique flavour with red pepper purée sauce. \nAlso had the duck confit pizza, with sweet gingered pear and Gorgonzola cheese, perfect combination of flavours. M